## This notebook inputs the missing values using KNN

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.impute import KNNImputer
from sklearn import preprocessing

#### Choose wether the imputation should be done on the data set with daily or weekly granularity

In [2]:
#INPUT, OUTPUT = 'CompleteIndexes.csv','KNNImputed.csv'
INPUT, OUTPUT = 'CompleteWeeklyIndexes', 'KNNWeeklyImputed'

In [3]:
df = pd.read_csv(INPUT+'_training_FE.csv', parse_dates=True, index_col=0)
df_test = pd.read_csv(INPUT+'_test_FE.csv', parse_dates=True, index_col=0)

In [4]:
df_test.head()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,vix_ROC2_13_52,vix_ROC2_13_52_diff1_period3,vix_ROC2_5_26,vix_ROC2_5_26_diff1_period3,vix_ROC2_13_26,vix_ROC2_13_26_diff1_period3,vix_52_Week_High,vix_52_Week_High_diff1_period3,vix_52_Week_Low,vix_52_Week_Low_diff1_period3
2015-01-02,2058.199951,2.708700e+09,17.790001,0.0,17832.990234,76270000.0,4230.240234,1.435150e+09,NaN,NaN,...,49.621539,-27.670303,49.621539,-27.670303,49.621539,-27.670303,-0.211951,-0.169688,0.505593,-0.169688
2015-01-09,2044.810059,3.364140e+09,17.549999,0.0,17737.369141,93390000.0,4213.279785,1.715830e+09,17197.730469,155200.0,...,47.603019,8.915046,-25.299152,-63.987126,47.603019,8.915046,-0.225534,0.062300,0.492010,0.062300
2015-01-16,2019.420044,4.056410e+09,20.950001,0.0,17511.570313,140480000.0,4142.140137,1.970520e+09,16864.160156,171100.0,...,76.198495,127.853667,44.482766,NaN,76.198495,NaN,-0.048449,0.367990,0.669095,0.367990
2015-01-23,2051.820068,3.573560e+09,16.660000,0.0,17672.599609,97110000.0,4278.140137,1.653830e+09,17511.750000,132500.0,...,40.117746,-9.503793,-31.992797,-81.614336,40.117746,-9.503793,-0.277577,-0.065626,0.439967,-0.065626
2015-01-30,1994.989990,4.568650e+09,20.969999,0.0,17164.949219,168560000.0,4148.430176,2.264230e+09,17674.390625,173500.0,...,76.366686,28.763667,25.870342,51.169495,76.366686,28.763667,-0.047495,0.178039,0.670049,0.178039


In [5]:
#Scaled data has zero mean and unit variance
scaler = preprocessing.StandardScaler()
scaler.fit(df)
array = scaler.transform(df)
array_test = scaler.transform(df_test)
df_scaled = pd.DataFrame(data=array, index=df.index, columns=df.columns)
df_test_scaled = pd.DataFrame(data=array_test, index=df_test.index, columns=df_test.columns)
df_scaled.head()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,vix_ROC2_13_52,vix_ROC2_13_52_diff1_period3,vix_ROC2_5_26,vix_ROC2_5_26_diff1_period3,vix_ROC2_13_26,vix_ROC2_13_26_diff1_period3,vix_52_Week_High,vix_52_Week_High_diff1_period3,vix_52_Week_Low,vix_52_Week_Low_diff1_period3
2000-01-07,0.588334,-1.002211,0.099912,0.0,0.020227,-0.443957,1.707031,-0.537214,1.945019,-1.565826,...,NaN,NaN,NaN,NaN,NaN,NaN,1.780223,NaN,-1.078036,NaN
2000-01-14,0.676196,-1.085193,-0.125931,0.0,0.109104,0.336040,1.914791,-0.494165,2.196016,-1.565826,...,NaN,NaN,NaN,NaN,NaN,NaN,1.413012,NaN,-1.078036,NaN
2000-01-21,0.587926,-1.011385,0.001243,0.0,-0.099881,-0.244602,2.087059,0.035615,2.170210,-1.565826,...,NaN,NaN,NaN,NaN,NaN,NaN,1.624272,NaN,-0.879743,NaN
2000-01-28,0.286643,-1.079295,0.584489,0.0,-0.327300,-0.461189,1.608014,-0.574034,2.353305,-1.565826,...,NaN,NaN,NaN,NaN,NaN,NaN,1.780223,-0.000104,-0.092655,1.516419
2000-02-04,0.524886,-1.109498,0.080178,0.0,-0.227555,-0.618274,2.116016,-0.306059,2.461300,-1.565826,...,NaN,NaN,NaN,NaN,NaN,NaN,1.066952,-0.536252,-0.762148,0.478210


In [6]:
imp = KNNImputer(n_neighbors=5, weights="uniform")
imp.fit(df_scaled)
df_imputed = imp.transform(df_scaled)
print(df_imputed.shape)
df_test_imputed = imp.transform(df_test_scaled)
df_final = pd.DataFrame(data=df_imputed, index=df.index, columns=df.columns)
df_test_final = pd.DataFrame(data=df_test_imputed, index=df_test.index, columns=df_test.columns)

(782, 412)


In [7]:
df_final.tail()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,vix_ROC2_13_52,vix_ROC2_13_52_diff1_period3,vix_ROC2_5_26,vix_ROC2_5_26_diff1_period3,vix_ROC2_13_26,vix_ROC2_13_26_diff1_period3,vix_52_Week_High,vix_52_Week_High_diff1_period3,vix_52_Week_Low,vix_52_Week_Low_diff1_period3
2014-11-28,2.911367,-0.240038,-0.819907,0.0,2.816486,-1.438162,2.665750,-1.799580,1.703751,0.483228,...,-0.205974,0.027095,-0.626575,0.046395,-0.639982,0.028687,-0.064434,0.090557,-0.327540,0.073523
2014-12-05,2.940345,0.305022,-0.977778,0.0,2.874378,-1.451109,2.634646,-0.275012,1.855242,0.205623,...,-0.455216,-0.342670,-1.047484,-0.417535,-0.951328,-0.372577,-0.485714,-0.644217,-0.722963,-0.616796
2014-12-12,2.669338,0.744673,0.029747,0.0,2.573736,-1.043260,2.501441,-0.033442,1.674719,1.382009,...,1.568718,2.425619,2.370445,3.055732,1.576916,2.631539,1.624479,2.803728,1.257714,2.622546
2014-12-19,2.922831,2.119495,-0.473467,0.0,2.806092,1.067770,2.599307,2.742025,1.756885,0.850162,...,1.087129,1.696677,1.557159,2.141156,0.975328,1.840500,0.719525,1.214480,0.408302,1.129448
2014-12-26,2.990064,-0.698382,-0.691636,0.0,2.916471,-1.703778,2.637635,-1.935233,1.821862,-0.377072,...,-0.039916,0.533681,1.138968,1.536870,0.569362,1.206619,0.245599,1.132916,-0.036535,1.052819


In [8]:
# Descaling the data
array_2 = scaler.inverse_transform(df_final)
df_final2 = pd.DataFrame(data=array_2, index=df.index, columns=df.columns)

array_test_2 = scaler.inverse_transform(df_test_final)
df_test_final2 = pd.DataFrame(data=array_test_2, index=df_test.index, columns=df_test.columns)

df_final2.head()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,vix_ROC2_13_52,vix_ROC2_13_52_diff1_period3,vix_ROC2_5_26,vix_ROC2_5_26_diff1_period3,vix_ROC2_13_26,vix_ROC2_13_26_diff1_period3,vix_52_Week_High,vix_52_Week_High_diff1_period3,vix_52_Week_Low,vix_52_Week_Low_diff1_period3
2000-01-07,1441.469971,1.225200e+09,21.719999,0.0,11522.559570,184900000.0,3529.600098,1.634930e+09,18193.410156,0.0,...,36.805798,13.575316,16.965588,-2.676683,36.805798,13.575316,0.000000,-0.023708,-5.551115e-17,0.119871
2000-01-14,1465.150024,1.085900e+09,19.660000,0.0,11722.980469,266830000.0,3704.739990,1.656630e+09,18956.550781,0.0,...,26.654561,5.299455,6.062116,-18.431609,26.654561,5.299455,-0.099647,-0.026051,-5.551115e-17,0.028136
2000-01-21,1441.359985,1.209800e+09,20.820000,0.0,11251.709961,205840000.0,3849.959961,1.923680e+09,18878.089844,0.0,...,28.138924,0.552724,8.484733,-10.280592,28.138924,0.552724,-0.042319,-0.022702,5.732795e-02,-0.013440
2000-01-28,1360.160034,1.095800e+09,26.139999,0.0,10738.870117,183090000.0,3446.129883,1.616370e+09,19434.779297,0.0,...,17.108280,23.520870,12.815777,10.987753,17.108280,23.520870,0.000000,0.000000,2.848806e-01,0.284881
2000-02-04,1424.369995,1.045100e+09,21.540001,0.0,10963.799805,166590000.0,3874.370117,1.751450e+09,19763.130859,0.0,...,35.794663,73.960382,19.353541,57.519260,35.794663,73.960382,-0.193555,-0.093908,9.132560e-02,0.091326


In [9]:
df_final2.to_csv(OUTPUT+'_training_FE.csv', index = True)
df_test_final2.to_csv(OUTPUT+ '_test_FE.csv', index = True)